In [1]:
import random
import numpy as np
import tensorflow as tf

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt',
        'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(filepath, 'rb').read().decode(encoding='utf-8')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
text = text[300000:800000]

In [5]:
characters = sorted(set(text))

char_to_index = dict((c, i) for i, c in enumerate(characters))
index_to_char = dict((i, c) for i, c in enumerate(characters))

In [6]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = []
next_char = []

In [7]:
for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i + SEQ_LENGTH])
    next_char.append(text[i + SEQ_LENGTH])

In [9]:
x = np.zeros((len(sentences), SEQ_LENGTH,
              len(characters)), dtype=bool)
y = np.zeros((len(sentences),
              len(characters)), dtype=bool)

for i, satz in enumerate(sentences):
    for t, char in enumerate(satz):
        x[i, t, char_to_index[char]] = 1
    y[i, char_to_index[next_char[i]]] = 1

In [10]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Activation, Dense, LSTM

In [11]:
model = Sequential()
model.add(LSTM(128,
               input_shape=(SEQ_LENGTH,
                            len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
model.compile(loss='categorical_crossentropy')

model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 55s 83ms/step - loss: 2.9244
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 62s 95ms/step - loss: 2.3187
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 55s 84ms/step - loss: 2.1813
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 66s 101ms/step - loss: 2.0914


In [16]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [17]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x_predictions = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, char in enumerate(sentence):
            x_predictions[0, t, char_to_index[char]] = 1

        predictions = model.predict(x_predictions, verbose=0)[0]
        next_index = sample(predictions,
                                 temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [19]:
print(generate_text(300, 0.2))
print(generate_text(300, 0.4))
print(generate_text(300, 0.5))
print(generate_text(300, 0.6))
print(generate_text(300, 0.7))
print(generate_text(300, 0.8))

nd?

ratcliff:
thomas the earl of surrey the corest and and the hard in the heres.

froule mery:
what the hard the sord the counde the beater of the merest.

our the reare:
and the here the beather sould and the hard the seare.

werent:
and with the beathere sour the store sore.

will there and in the beather the seares the sore.

there a
eeing ill.
thy death-bed is no lesser that the will the merd thou bould here here her.

our cordoug:
add with searet the mery to the the soures thes and tore parking there and and thee to herch and the herd with the ground of stare.

weachire:
thear the preast the tore the cander and and of thas of i steres.

condis:
so me weren wist rome
ve saffron to colour the warden
pies; mare and and beather ford chand here in wilchersing
thee sond, i the entise were stit that to be tie there.

toree to sour, my rimen, brow the lofdongelas,
and were the are the sarce to the forthy my wile shere
so lord wither warke her thou thing core hare hith the thest thee the